# 2. How Is Service Request Distribution by Zip Code?

#### Methodology

1. Import map data using a GeoJSON file and match zip codes.
2. Identify area with the most service requests.
3. Visualize zip code by service request type on the map.

#### Import Libraries and Prepare the Dataset
- Imported the original data as 'df_original' and I created Dataframe 'df' which containing only necessary columns.
- Changed data type of 'Created Date' to datetime format and created a new column named 'date'.


In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt  

from matplotlib.ticker import PercentFormatter, FuncFormatter

df_original = pd.read_csv(r'C:\Users\Dell\DA_FILE\100_My_Project\8_Service_request\Customer_Service_Requests.csv', dtype={12 : object})

sns.set_style('ticks')

# Remove unnecessary columns
df = df_original[['Service Request Number', 'Service Request Type', 'City Department',
       'Created Date', 'Method Received', 'Status',  'ZIP Code']].copy()
df.loc[:,'date'] =pd.to_datetime(df['Created Date'])

#### Import `Plotly` and GeoJSON file for Map Visualization
I obtained GeoJSON data from [here](https://github.com/seattleio/seattle-boundaries-data/blob/master/data/zip-codes.geojson), I used zip code as matching key.

In [3]:
import plotly.express as px
import json

with open(r'C:\Users\Dell\DA_FILE\100_My_Project\8_Service_request\zip-codes.geojson', 'r') as f:
    geojson_data = json.load(f)
print(geojson_data['features'][0]['properties']) 
# validate property fields  :GEOID10

{'ZCTA5CE10': '98136', 'AFFGEOID10': '8600000US98136', 'GEOID10': '98136', 'ALAND10': 5954030.0, 'AWATER10': 2163632.0}


### 1. Explore Total Request Count per Zip Code
To find the highest number of request area, I grouped the dataset by zip code and counted total service requests, then sorted value by total count.

In [4]:
df_zip = df.groupby('ZIP Code')['Service Request Number'].count().reset_index().sort_values(by='Service Request Number',ascending=False)

Plot the total count of service request data using `choropleth_map`. 
I matched the key using `ZIP Code`, set color as dark red for most requested zip code area.

In [5]:
fig = px.choropleth_map(
    df_zip, 
    geojson=geojson_data, 
    locations='ZIP Code', 
    title='Service Request Distribution in SEATTLE',
    color='Service Request Number', # color based on request count
    featureidkey='properties.GEOID10',# matching key
    color_continuous_scale="YlOrRd", #set most requested place as Red.
    map_style="carto-positron",
    zoom=9.5, center = {"lat": 47.61, "lon": -122.33}, # show Seattle
    opacity=0.8
     )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0}, 
                  coloraxis_colorbar=dict(title="Total Request Count"),
                  title={'xanchor': 'left', 'font': {'size': 24}},
                  width=600, height=600 )
fig.show()

![Service Request Distribution in SEATTLE](image/seattle_distribution.png)

#### Explore Top 5 Most Requested Service Types by Zip Code.
I created a list of top 5 requested service types.

In [13]:
# Create list of top 5 requested type
top5_type = df['Service Request Type'].value_counts().index[:5]

In order to visualize top 5 requested type in maps, I filtered data by type, then created a pivot table shows service requested count by zip code, finally plot the map a separate `choropleth_map`.

In [11]:
df_type = df[df['Service Request Type'].isin(top5_type)]
df_type_pivot = df_type.pivot_table(index='ZIP Code', columns='Service Request Type', aggfunc='count', values='Service Request Number')

In [12]:
for type in top5_type:
    fig = px.choropleth_map(
        df_type_pivot, 
        geojson=geojson_data, 
        locations=df_type_pivot.index, 
        color=type, # color based on request count
        title=type,
        featureidkey='properties.GEOID10',# matching key
        color_continuous_scale="YlOrRd", #set most requested place as Red.
        #range_color=(0, 15300), # service request total count range
        map_style="carto-positron",
        zoom=9.5, center = {"lat": 47.61, "lon": -122.33}, # show Seattle
        opacity=0.8
        )

    fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0},
                      title={'xanchor': 'left', 'font': {'size': 24}},
                      coloraxis_colorbar=dict(title="Request Count"),
                      width=600, height=600 )
    fig.show()

![Abandoned Vehicle/72hr Parking Ordinance](image/1_longterm.png)
![Parking Enforcement](image/2_shortterm.png)
![Unauthorized Encampment](image/3_encampment.png)

#### Insight
- Long-term parking is the issue across Seattle.
- Short-term parking issues are especially common in zip code 98103 and 98122.
- Unauthorized Encampment is problem especially in zip code 98107.
- Illegal dumping/ needles are more frequent in South Seattle.
- Graffiti reports are more common in downtown area.